In [1]:
import geopandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import panel as pn
import ipywidgets as ipw
import geopandas as gpd

import json
import panel.widgets as pnw

import itertools
import warnings
import hvplot

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer

pn.extension()

In [2]:
import pandas as pd
import geopandas as gpd
import json
import matplotlib as mpl
import pylab as plt

from bokeh.io import output_file, show, output_notebook, export_png
from bokeh.models import ColumnDataSource, GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.plotting import figure
from bokeh.palettes import brewer

import panel as pn
import panel.widgets as pnw

import panel as pn
import hvplot.pandas
import pandas as pd
import numpy as np
import panel as pn
pn.extension()

In [3]:
# Geo-data
world = gpd.read_file(geopandas.datasets.get_path("naturalearth_lowres"))

# Cause of death - data
url  = 'https://raw.githubusercontent.com/Denze8/Data-Visualization-Project/main/Data/full_data_geo.csv'
data = pd.read_csv(url)

data = data.drop(columns = ['geometry', 'continent', 'Country.Territory'])


# 
geo_data = world.merge(data, how = "right", left_on = ['iso_a3'], right_on = ['Code'])
geo_data['Year'] = geo_data['Year'].values.astype(int)

/var/folders/k6/14q6_b813yncx2fqg44h95_r0000gn/T/ipykernel_70259/2803143146.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  world = gpd.read_file(geopandas.datasets.get_path("naturalearth_lowres"))


In [4]:
x1 = data.Code.unique()
world = world.query('iso_a3 in @x1')


In [5]:
def get_geodatasource(gdf):    
    """Get getjsondatasource from geopandas object"""
    json_data = json.dumps(json.loads(gdf.to_json()))
    return GeoJSONDataSource(geojson = json_data)

In [6]:
def bokeh_plot_map(gdf, column = None, country_select = None, title = ''):
    gdf = gdf.copy()
    countries = list(gdf.name)
    gdf["colors"] = ['#ff0000' if i in country_select else '#000000' for i in countries]
    gdf["size"]   = [1.5 if i in country_select else 0.5 for i in countries]
    geosource = get_geodatasource(gdf)
    palette = brewer['OrRd'][8]
    palette = palette[::-1]
    vals = gdf[column]
    
    #Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = vals.min(), high = vals.max())
    color_bar    = ColorBar(color_mapper = color_mapper, label_standoff = 8, width = 500, height = 20, 
                         location=(0,0), orientation='horizontal')
    
    tools = 'wheel_zoom, pan, reset, hover'
    TOOLTIPS = [("Country", "@name"), ("Population", "@Population"), ("Deaths", "".join(['@', column]))]
    p = figure(title = title, height = 400 , width = 850, toolbar_location = 'right', tools = tools, tooltips = TOOLTIPS)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    
    #Add patch renderer to figure
    p.patches('xs','ys', source = geosource, fill_alpha = 1, line_width = 'size', line_color = 'colors',  
              fill_color={'field' :column , 'transform': color_mapper})
    
    #Specify figure layout.
    p.add_layout(color_bar, 'below')
    return p

In [37]:
def map_dash():

    from bokeh.models.widgets import DataTable
    # Panels
    map_pane    = pn.pane.Bokeh(width = 500, height = 400)
    bar         = pn.pane.HoloViews()
    
    # Widgets
    data_select    = pnw.Select(name='Cause of Death', value = 'Meningitis', options= list(geo_data.columns)[8:-2])
    country_select = pnw.MultiSelect(name='Countries', value = ['United States of America', 'Sweden', 'Russia'], options =  [i for i in np.unique(geo_data['name'])])
    year_slider    = pnw.RangeSlider(name='Year', start=1990, end=2019, value=(1990, 2019), step = 1)
    checkbox       = pnw.Checkbox(name='Continents', value = True)
    toggle_group   = pnw.ToggleGroup(name='Continents', value = ['Africa'], options = list(geo_data.continent.unique()))
    
    def update_map(event):
        
        continents = toggle_group.value
        geo_data_ = geo_data.query('continent in @continents')
        
        geo_ = geo_data_.loc[(geo_data['Year'] == year_slider.value[0])]
        map_pane.object = bokeh_plot_map(geo_, data_select.value, country_select.value)
        
        gdata = geo_data.loc[(geo_data['Year'] >= year_slider.value[0]) & (geo_data['Year'] <= year_slider.value[1])]
        countries = country_select.value
        gdata = gdata.query('name in @countries').pivot(index = 'Year', columns = 'name', values = data_select.value)
        bar.object = gdata.hvplot.bar()
        return 
    
    
    year_slider.param.watch(update_map,'value')
    year_slider.param.trigger('value')
    
    data_select.param.watch(update_map, 'value')
    
    country_select.param.trigger('value')
    country_select.param.watch(update_map, 'value')
    toggle_group.param.watch(update_map, 'value')
    toggle_group.param.trigger('value')
    
    app = pn.Column(toggle_group,
            pn.Row(
                pn.Column(
                    country_select,
                    year_slider,
                    data_select
                    ), 
                map_pane),
        bar)
    return app

map_dash()

Column
    [0] CheckButtonGroup(name='Continents', options=['Oceania', 'Africa', ...], value=['Africa'])
    [1] Row
        [0] Column
            [0] MultiSelect(name='Countries', options=['Afghanistan', ...], value=['United States of America...])
            [1] RangeSlider(end=2019, name='Year', start=1990, step=1, value=(1990, 2019), value_end=2019, value_start=1990)
            [2] Select(name='Cause of Death', options=['Meningitis', ...], value='Meningitis')
        [1] Bokeh(figure, align='auto', height=400, width=500)
    [2] HoloViews(Bars, height=300, sizing_mode='fixed', width=700)

In [9]:
# Kode til flere år i stedet for 1 
cols = list(itertools.chain.from_iterable([list(geo_data.columns[1:7]), ['Population']]))
simple                    = geo_data.loc[geo_data['Year'] == 1990][cols].sort_values('name')
simple[data_select.value] = list(geo_data.groupby('name')[data_select.value].aggregate('sum'))
map_pane.object           = bokeh_plot_map(simple, data_select.value, country_select.value)


['continent', 'name', 'iso_a3', 'gdp_md_est', 'geometry', 'Code', 'Population']